# 10 abr 2018

Ontem nós conseguimos migrar o dataset de candidatos de 2016 para tabela.

Hoje queremos aprimorar o processo **criando parsers** para cada um dos itens da tabela candidato e aplicar as mudanças para popular o banco com os dados dos anos anteriores a 2016

In [1]:
import pandas as pd
import numpy as np
import mysql.connector as mysql

In [2]:
import os
import glob

from utils.tse_data_headers import get_header

Configurações para o banco de dados local

In [3]:
bd_config = {
    'user': 'root',
    'password': '',
    'host': '127.0.0.1',
    'database': 'laddres'
}

In [6]:
df = pd.read_csv(
    '../data/2018-04-04-candidaturas-tse.xz',
    low_memory=False
)

In [4]:
# DIRETORIO_TEMPORARIO = mkdtemp()
DIRETORIO_TEMPORARIO = os.path.join('../temp')

# Nome e endereço dos arquivos do diretório do TSE
PREFIXO_ARQUIVO_CANDIDATURAS = 'consulta_cand_'

In [5]:
import sys
import time

import utils.tse_parsers as parsers

def migrar_dados(ano):
    tempo_inicio_ano = time.time()

    arquivos = PREFIXO_ARQUIVO_CANDIDATURAS + str(ano) + '*.txt'
    pathes = os.path.join(DIRETORIO_TEMPORARIO, arquivos)
    arquivos_do_ano = sorted(glob.glob(pathes))

    for arquivo_estado in arquivos_do_ano:
        tempo_inicio_estado = time.time()
        print('Iniciando ', arquivo_estado)

        df_estado = pd.read_csv(
            arquivo_estado,
            sep = ';',
            header = None,
            dtype = np.str,
            names = get_header(ano),
            encoding = 'iso-8859-1'
        )

        candidatos = (
            df_estado[[
                'NOME_CANDIDATO',
                'DATA_NASCIMENTO',
                'CPF_CANDIDATO',
                'NUM_TITULO_ELEITORAL_CANDIDATO',
#                 'NM_EMAIL',
                'NOME_MUNICIPIO_NASCIMENTO',
                'SIGLA_UF_NASCIMENTO',
                'CODIGO_NACIONALIDADE',
                'COD_GRAU_INSTRUCAO',
                'CODIGO_OCUPACAO',
            ]]
            .drop_duplicates('CPF_CANDIDATO', keep='last')
            .rename(columns = {
                'NOME_CANDIDATO': 'nome',
                'DATA_NASCIMENTO': 'data_nascimento',
                'CPF_CANDIDATO': 'cpf',
                'NUM_TITULO_ELEITORAL_CANDIDATO': 'titulo_eleitoral',
#                 'NM_EMAIL': 'email',
                'NOME_MUNICIPIO_NASCIMENTO': 'cidade',
                'SIGLA_UF_NASCIMENTO': 'estado',
                'CODIGO_NACIONALIDADE': 'nacionalidade',
                'COD_GRAU_INSTRUCAO': 'grau_instrucao',
                'CODIGO_OCUPACAO': 'ocupacao'
            })
        )

        total_candidatos = candidatos.shape[0]
        print('Total de candidatos em', ano, ':', total_candidatos)


        cnx = mysql.connect(**bd_config)
        cursor = cnx.cursor(buffered=True)

        for candidato in candidatos.itertuples():
            # Buscar id da cidade
            cidade = parsers.parse_cidade(candidato.cidade)
            sigla_estado = parsers.parse_estado(candidato.estado)
            try:
                query = ("SELECT id FROM cidade WHERE "
                     "nome = \"" + cidade + "\" AND "
                     "estado_id = (SELECT id FROM estado WHERE sigla = '" + sigla_estado + "')")
            except:
                print('Candidato: {}'.format(candidato))
                print('Cidade: {}'.format(cidade))
                print('Sigla do Estado: {}'.format(sigla_estado))
                cursor.close()
                cnx.close()
                raise

            try:
                cursor.execute(query)
            except:
                print(candidato)
                print(query)
                cursor.close()
                cnx.close()
                raise

            cidades = cursor.fetchall()
            cidade_id = cidades[0][0] if len(cidades) > 0 else None

            # Cadastrar candidato
            novo_candidato = ("INSERT INTO candidato" + str(ano) + " "
                           "(nome, data_nascimento, cpf, titulo_eleitoral, email, cidade_id, ocupacao_id, nacionalidade_id, grau_instrucao_id) "
                           "VALUES (%(nome)s, %(data_nascimento)s, %(cpf)s, %(titulo_eleitoral)s, %(email)s, %(cidade_id)s, %(ocupacao)s, %(nacionalidade)s, %(grau_instrucao)s)")
            atributos = {
                'nome': parsers.parse_nome(candidato.nome),
                'data_nascimento': parsers.parse_data(candidato.data_nascimento),
                'cpf': parsers.parse_cpf(candidato.cpf),
                'titulo_eleitoral': parsers.parse_titulo_eleitoral(candidato.titulo_eleitoral),
                'email': None,
#                 'email': parsers.parse_email(candidato.email),
                'cidade_id': cidade_id,
                'ocupacao': parsers.parse_ocupacao(candidato.ocupacao),
                'nacionalidade': parsers.parse_nacionalidade(candidato.nacionalidade),
                'grau_instrucao': parsers.parse_grau_instrucao(candidato.grau_instrucao)
            }

            try:
                cursor.execute(novo_candidato, atributos)
                cnx.commit()
            except:
                print(candidato)
                cursor.close()
                cnx.close()
                raise

            sys.stdout.write('\r{} de {}'.format(candidato[0], total_candidatos))
            sys.stdout.flush()

        cursor.close()
        cnx.close()

        sys.stdout.write('\n')
        tempo_final_estado = time.time()
        print('Tempo total desse estado: {:.2f}s \n'.format((tempo_final_estado - tempo_inicio_estado)))

    tempo_final_ano = time.time()
    print('Tempo total em {}: {:.2f}s \n'.format(ano, (tempo_final_ano - tempo_inicio_ano)))


In [16]:
migrar_dados(2014)

Iniciando  ../temp\consulta_cand_2014_AC.txt
Total de candidatos em 2014 : 617
620 de 617
Tempo total desse estado:  2.4361228942871094 

Iniciando  ../temp\consulta_cand_2014_AL.txt
Total de candidatos em 2014 : 474
478 de 474
Tempo total desse estado:  1.8667545318603516 

Iniciando  ../temp\consulta_cand_2014_AM.txt
Total de candidatos em 2014 : 763
785 de 763
Tempo total desse estado:  2.938457489013672 

Iniciando  ../temp\consulta_cand_2014_AP.txt
Total de candidatos em 2014 : 546
549 de 546
Tempo total desse estado:  2.0768895149230957 

Iniciando  ../temp\consulta_cand_2014_BA.txt
Total de candidatos em 2014 : 1057
1092 de 1057
Tempo total desse estado:  4.516520977020264 

Iniciando  ../temp\consulta_cand_2014_BR.txt
Total de candidatos em 2014 : 23
27 de 23
Tempo total desse estado:  0.11257529258728027 

Iniciando  ../temp\consulta_cand_2014_CE.txt
Total de candidatos em 2014 : 873
878 de 873
Tempo total desse estado:  3.53635573387146 

Iniciando  ../temp\consulta_cand_2014

TypeError: 'str' object is not callable

In [7]:
migrar_dados(2010)

Iniciando  ../temp\consulta_cand_2010_AC.txt
Total de candidatos em 2010 : 455
455 de 455
Tempo total desse estado:  1.7400455474853516 

Iniciando  ../temp\consulta_cand_2010_AL.txt
Total de candidatos em 2010 : 462
470 de 462
Tempo total desse estado:  1.761220932006836 

Iniciando  ../temp\consulta_cand_2010_AM.txt
Total de candidatos em 2010 : 533
534 de 533
Tempo total desse estado:  2.0145273208618164 

Iniciando  ../temp\consulta_cand_2010_AP.txt
Total de candidatos em 2010 : 399
486 de 399
Tempo total desse estado:  1.5309131145477295 

Iniciando  ../temp\consulta_cand_2010_BA.txt
Total de candidatos em 2010 : 1013
1019 de 1013
Tempo total desse estado:  4.267146348953247 

Iniciando  ../temp\consulta_cand_2010_BR.txt
Total de candidatos em 2010 : 18
21 de 18
Tempo total desse estado:  0.09000205993652344 

Iniciando  ../temp\consulta_cand_2010_CE.txt
Total de candidatos em 2010 : 799
806 de 799
Tempo total desse estado:  3.0926225185394287 

Iniciando  ../temp\consulta_cand_20

TypeError: 'str' object is not callable

In [10]:
migrar_dados(2012)

Iniciando  ../temp\consulta_cand_2012_AC.txt
Total de candidatos em 2012 : 2338
2342 de 2338
Tempo total desse estado: 8.82 

Iniciando  ../temp\consulta_cand_2012_AL.txt
Total de candidatos em 2012 : 7773
7820 de 7773
Tempo total desse estado: 29.77 

Iniciando  ../temp\consulta_cand_2012_AM.txt
Total de candidatos em 2012 : 7721
7798 de 7721
Tempo total desse estado: 29.10 

Iniciando  ../temp\consulta_cand_2012_AP.txt
Total de candidatos em 2012 : 1916
1958 de 1916
Tempo total desse estado: 7.31 

Iniciando  ../temp\consulta_cand_2012_BA.txt
Total de candidatos em 2012 : 36167
36354 de 36167
Tempo total desse estado: 150.21 

Iniciando  ../temp\consulta_cand_2012_CE.txt
Total de candidatos em 2012 : 14588
14736 de 14588
Tempo total desse estado: 55.92 

Iniciando  ../temp\consulta_cand_2012_ES.txt
Total de candidatos em 2012 : 9700
9806 de 9700
Tempo total desse estado: 36.92 

Iniciando  ../temp\consulta_cand_2012_GO.txt
Total de candidatos em 2012 : 20467
20635 de 20467
Tempo tota

ValueError: cannot switch from automatic field numbering to manual field specification

In [7]:
migrar_dados(2006)

Iniciando  ../temp\consulta_cand_2006_AC.txt
Total de candidatos em 2006 : 462
463 de 462
Tempo total desse estado: 1.77 

Iniciando  ../temp\consulta_cand_2006_AL.txt
Total de candidatos em 2006 : 393
395 de 393
Tempo total desse estado: 1.52 

Iniciando  ../temp\consulta_cand_2006_AM.txt
Total de candidatos em 2006 : 561
560 de 561
Tempo total desse estado: 2.22 

Iniciando  ../temp\consulta_cand_2006_AP.txt
Total de candidatos em 2006 : 386
385 de 386
Tempo total desse estado: 1.46 

Iniciando  ../temp\consulta_cand_2006_BA.txt
Total de candidatos em 2006 : 883
888 de 883
Tempo total desse estado: 3.73 

Iniciando  ../temp\consulta_cand_2006_BR.txt
Total de candidatos em 2006 : 16
19 de 16
Tempo total desse estado: 0.08 

Iniciando  ../temp\consulta_cand_2006_CE.txt
Total de candidatos em 2006 : 784
787 de 784
Tempo total desse estado: 3.09 

Iniciando  ../temp\consulta_cand_2006_DF.txt
Total de candidatos em 2006 : 863
869 de 863
Tempo total desse estado: 3.38 

Iniciando  ../temp\

In [6]:
migrar_dados(2004)

Iniciando  ../temp\consulta_cand_2004_AC.txt
Total de candidatos em 2004 : 1503
1510 de 1503
Tempo total desse estado: 5.64s 

Iniciando  ../temp\consulta_cand_2004_AL.txt
Total de candidatos em 2004 : 7007
7040 de 7007
Tempo total desse estado: 25.73s 

Iniciando  ../temp\consulta_cand_2004_AM.txt
Total de candidatos em 2004 : 6098
6146 de 6098
Tempo total desse estado: 22.74s 

Iniciando  ../temp\consulta_cand_2004_AP.txt
Total de candidatos em 2004 : 1238
1239 de 1238
Tempo total desse estado: 4.66s 

Iniciando  ../temp\consulta_cand_2004_BA.txt
Total de candidatos em 2004 : 30513
30699 de 30513
Tempo total desse estado: 125.24s 

Iniciando  ../temp\consulta_cand_2004_CE.txt
Total de candidatos em 2004 : 11911
11961 de 11911
Tempo total desse estado: 62.39s 

Iniciando  ../temp\consulta_cand_2004_ES.txt
Total de candidatos em 2004 : 7989
8015 de 7989
Tempo total desse estado: 30.58s 

Iniciando  ../temp\consulta_cand_2004_GO.txt
Total de candidatos em 2004 : 17107
17165 de 17107
Tem